In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

## Prepare the data

In [2]:
players_df = pd.read_csv("../data/players.csv")
sessions_df = pd.read_csv("../data/sessions.csv")
tournaments_df = pd.read_csv("../data/tournaments.csv")
transactions_df = pd.read_csv("../data/transactions.csv")

# Convert date columns to datetime
players_df['signup_date'] = pd.to_datetime(players_df['signup_date'])

sessions_df['start_ts'] = pd.to_datetime(sessions_df['start_ts'])
sessions_df['end_ts'] = pd.to_datetime(sessions_df['end_ts'])

tournaments_df['start_ts'] = pd.to_datetime(tournaments_df['start_ts'])

transactions_df['txn_ts'] = pd.to_datetime(transactions_df['txn_ts'])

In [3]:
deposits_mask = transactions_df['txn_type'] == "deposit"
deposits_df = transactions_df[deposits_mask][['player_id', 'txn_ts', 'amount']].copy()
deposits_df.rename(columns={'txn_ts': 'ts', 'amount': 'deposit'}, inplace=True)

In [4]:
deposits_df.head()

,player_id,ts,deposit
2,1,2024-12-21 02:28:26,90.73
7,2,2025-03-16 01:39:12,20.36
9,3,2025-04-28 14:45:03,86.54
10,3,2025-04-27 13:11:03,85.79
14,4,2025-02-15 19:29:37,56.15


In [5]:
deposits_df['player_id'].value_counts()

2474     5
8057     5
15700    5
8070     5
10042    5
        ..
16433    1
10300    1
8253     1
14398    1
4098     1
Name: player_id, Length: 12026, dtype: int64

##  Step 0: Baseline model

### predict the avrage 30 days deposites per player

In [6]:
# Step 1: Calculate the average deposit per player over the last 30 days (baseline)
# Sort the deposits by player_id and timestamp
deposits_df = deposits_df.sort_values(by=['player_id', 'ts'])

# Create a rolling window feature: average deposit over the last 30 days per player
deposits_df['30_day_avg_deposit'] = deposits_df.groupby('player_id')['deposit'].rolling(window=30, min_periods=1).mean().reset_index(0, drop=True)

# Step 2: Use the 30-day rolling average as the target variable for our baseline model
baseline_df = deposits_df[['player_id', 'ts', '30_day_avg_deposit']].dropna()

# Step 3: Aggregate the data by player to create a single row per player with the baseline target
baseline_df = baseline_df.groupby('player_id').agg({'30_day_avg_deposit': 'mean'}).reset_index()

# Split into train and test sets
X = baseline_df[['player_id']]  # In a real model, we might add more features here, but for the baseline, we'll use player_id
y = baseline_df['30_day_avg_deposit']

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Use a simple model - we will use the mean of the training data as our prediction
# For baseline, the prediction for all players is simply the mean of the training set
baseline_prediction = y_train.mean()

# Step 6: Evaluate the baseline model using mean squared error
y_pred = [baseline_prediction] * len(y_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Baseline model MSE: {mse}")

Baseline model MSE: 597.3587036513112


In [7]:
deposits_df.sort_values(by=['player_id', 'ts'])

,player_id,ts,deposit,30_day_avg_deposit
2,1,2024-12-21 02:28:26,90.73,90.730000
7,2,2025-03-16 01:39:12,20.36,20.360000
10,3,2025-04-27 13:11:03,85.79,85.790000
9,3,2025-04-28 14:45:03,86.54,86.165000
14,4,2025-02-15 19:29:37,56.15,56.150000
...,...,...,...,...
49880,19997,2025-03-25 07:41:12,40.22,56.490000
49879,19997,2025-03-26 11:44:12,26.57,46.516667
49881,19998,2025-04-12 14:48:08,19.12,19.120000
49885,20000,2025-01-18 20:25:07,44.41,44.410000


### Feature Engineering

### Create Fact table per user

In [8]:
# 1. Aggregate transaction data (total deposits per player)
player_deposits = transactions_df.groupby('player_id').agg({'amount': 'sum'}).reset_index()
player_deposits.rename(columns={'amount': 'total_deposits'}, inplace=True)

# 2. Player-level features (country, acquisition_channel, signup_date)
players_df['signup_date'] = pd.to_datetime(players_df['signup_date'])
players_df['days_since_signup'] = (pd.to_datetime('today') - players_df['signup_date']).dt.days
player_features = players_df[['player_id', 'country', 'acquisition_channel', 'days_since_signup']]

# 3. Session-related features
# Calculate average session length per player (in seconds)
sessions_df['session_length'] = (pd.to_datetime(sessions_df['end_ts']) - pd.to_datetime(sessions_df['start_ts'])).dt.total_seconds()
avg_session_length = sessions_df.groupby('player_id').agg({'session_length': 'mean'}).reset_index()
avg_session_length.rename(columns={'session_length': 'avg_session_length'}, inplace=True)

# Calculate most active device per player
most_active_device = sessions_df.groupby(['player_id', 'device']).size().reset_index(name='device_count')
most_active_device = most_active_device.loc[most_active_device.groupby('player_id')['device_count'].idxmax()][['player_id', 'device']]
most_active_device.rename(columns={'device': 'most_active_device'}, inplace=True)

# 4. Merge all features into a single fact table
fact_table = player_features.merge(player_deposits, on='player_id', how='left') \
                            .merge(avg_session_length, on='player_id', how='left') \
                            .merge(most_active_device, on='player_id', how='left')

In [9]:
# Players that only deposited and weren't active
fact_table['visitor'] = fact_table['most_active_device'].isna().replace({True: 'yes', False: 'no'})

In [10]:
fact_table.head()

,player_id,country,acquisition_channel,days_since_signup,total_deposits,avg_session_length,most_active_device,visitor
0,1,CA,Facebook,195,160.32,2550.000000,Android,no
1,2,ZA,Organic,133,82.34,1573.333333,iOS,no
2,3,CA,Google,143,180.33,2208.000000,Android,no
3,4,GB,Referral,163,61.15,735.000000,iOS,no
4,5,FR,Facebook,164,10.00,NaN,NaN,yes


## Model 1: Time series model

In [13]:
deposits_df = deposits_df.sort_values(by=['player_id', 'ts'])
deposits_df

,player_id,ts,deposit,30_day_avg_deposit
2,1,2024-12-21 02:28:26,90.73,90.730000
7,2,2025-03-16 01:39:12,20.36,20.360000
10,3,2025-04-27 13:11:03,85.79,85.790000
9,3,2025-04-28 14:45:03,86.54,86.165000
14,4,2025-02-15 19:29:37,56.15,56.150000
...,...,...,...,...
49880,19997,2025-03-25 07:41:12,40.22,56.490000
49879,19997,2025-03-26 11:44:12,26.57,46.516667
49881,19998,2025-04-12 14:48:08,19.12,19.120000
49885,20000,2025-01-18 20:25:07,44.41,44.410000
